# Importing my own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import


In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/code/mybook-dashboard'

In [2]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
# from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [3]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
# mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))
mybooksgr = mybooksgr.sample(250)

In [4]:
len(mybooksgr)

250

In [5]:
import nest_asyncio
nest_asyncio.apply()

from apps.async_googleapi import book_info_add, asyncio
from apps.api import api_key
pimydf = asyncio.run(book_info_add(mybooksgr, api_key))

Blue Sisters
How to Win Friends and Influence People
A Girl Is a Half-Formed Thing
Conquest
Maame
Kala
Øst for Eden 1 (Øst for Eden #1)
Visual Intelligence: Sharpen Your Perception, Change Your Life
La Peregrina (Peregrino, #2)
Elskede barn
The Family Upstairs (The Family Upstairs, #1)
The Power of Now: A Guide to Spiritual Enlightenment
Litt redd, bare
The Little Coffee Shop of Kabul
Behold the Dreamers
Jar of Hearts
To Kill a Mockingbird
Jenny
Dominicana
The Book of Azrael (Gods and Monsters, #1)
Such a Fun Age
Ceremony
The Housemaid (The Housemaid, #1)
Local Woman Missing
It Ends with Us (It Ends with Us, #1)
Good Material
Gone Girl
Vinternoveller
From Blood and Ash (Blood and Ash, #1)
The da Vinci Code (Robert Langdon, #2)
The Dutch House
Daughter of the Moon Goddess (The Celestial Kingdom Duology #1)
Min europeiska familj: De senaste 54 000 åren
Eva Luna
Guns, Germs, and Steel: The Fates of Human Societies
Death on Cromer Beach (DS Ashley Knight #1)
The Magnolia Palace
Naked Stati

# Merge dataframes 
mybooks = pd.merge(mybooksgr,
                     pimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

In [6]:

from apps.dataimport import dataprep
nmyreads = dataprep(mybooksgr, pimydf)

In [7]:
import pickle
# nmyreads.to_pickle("assets/my_books.pkl")
nmyreads.to_parquet('assets/my_books.parquet', engine='pyarrow')

# Collect book topics 

In [8]:
# Collect topics for my own books from OLapi
from apps.collect_data import get_book_topics
my_topics = get_book_topics(mybooksgr)

In [9]:
dict = mybooksgr[['Author','Title']].to_dict()
list(set(dict['Title'].values()))
    

['These Silent Woods',
 'Sharp Objects',
 'Litt redd, bare',
 'It Ends with Us (It Ends with Us, #1)',
 'Blink: The Power of Thinking Without Thinking',
 'Less',
 'The Inmate',
 'Predictably Irrational: The Hidden Forces That Shape Our Decisions',
 'Skyggedanseren',
 'A Court of Wings and Ruin (A Court of Thorns and Roses, #3)',
 'Good Material',
 'The Nightingale',
 'The Power of Habit: Why We Do What We Do in Life and Business',
 'Looking for Bono (Twenty in 2020)',
 'The Housemaid (The Housemaid, #1)',
 'The Firm (The Firm, #1)',
 "There's No Such Thing as an Easy Job",
 'The Seven Husbands of Evelyn Hugo',
 'To Paradise',
 'May We Be Forgiven',
 'Cooper Bartholomew Is Dead',
 'Free Lunch Thinking: How Economics Ruins the Economy',
 'Shatter Me (Shatter Me, #1)',
 'The Kind Worth Killing (Henry Kimball/Lily Kintner, #1)',
 'The Unseen (The Barrøy Chronicles #1)',
 'Out',
 'Beautiful Malice',
 '1Q84 (1Q84, #3)',
 '1Q84 Book 1 (1Q84, #1)',
 'Pretty Girls',
 'Blue Sisters',
 'Penance',

In [10]:
df = pd.DataFrame(dict)
df

,Author,Title
20,Coco Mellors,Blue Sisters
89,Dale Carnegie,How to Win Friends and Influence People
320,Eimear McBride,A Girl Is a Half-Formed Thing
195,Nina Allan,Conquest
247,Jessica George,Maame
...,...,...
292,Gabriel García Márquez,One Hundred Years of Solitude
57,George R.R. Martin,"A Storm of Swords (A Song of Ice and Fire, #3)"
501,Cristina García,Monkey Hunting: A Novel (Ballantine Reader's C...
382,Trude Teige,Mormor danset i regnet


In [11]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)

In [12]:
mybooksgr.query('`Read Count` ==1').head(2).to_csv("assets/my_books_test.csv", index=False)